<a href="https://colab.research.google.com/github/icebearian/CS221-Natural-Language-Processing/blob/master/PosTaggingCapstoneProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Đề tài: **Tách từ và gán nhãn từ loại bằng Hidden Markov Model.**
---

# **Tạo ngữ liệu**
---
> Bao gồm **30** câu đã được gán nhãn từ loại:
- **25** câu dùng để train
- **5** câu dùng để làm dữ liệu **gold** cho quá trình tính toán độ chính xác.

In [38]:
gold_data = [('Alex', 'NNP'),('was', 'NNP'),('born', 'VB'),('in', 'NNP'),('California', 'NNP')]

In [39]:
train_data = [
              [('Alex', 'NNP'), ('was', 'NNP'), ('born', 'NNP'), ('in', 'NNP'), ('Connecticut', 'NNP'), ('.', 'NNP')],
              [('Joe', 'NNP'), ('met', 'VBD'), ('Joanne', 'NNP'), ('in', 'IN'), ('Delhi', 'NNP'), ('.', 'NNP')],
              [('Chicago', 'NNP'), ('is', 'VBZ'), ('the', 'DT'), ('birthpace', 'NNP'), ('of', 'NNP'), ('Ginny', 'NNP')]
              ]
#print ("Train data:", train_data)

# **GÁN NHÃN TỪ LOẠI (PART-OF-SPEECH Tagging)**
---



# **Huấn luyện dữ liệu**
---
Sử dụng mô hình **Hidden Markov Model** với phương pháp smoothing là **Laplace**.:


In [40]:
from nltk.tag.hmm import HiddenMarkovModelTrainer
from nltk.probability import LaplaceProbDist
trainer = HiddenMarkovModelTrainer()
tagger = trainer.train_supervised(train_data,estimator=LaplaceProbDist)

# **Đánh giá**
---
Sử dụng độ chính xác **(Accuracy)** bằng công thức:<br>
$Acc = \frac{n}{N}$<br>
- $n$ là số lượng nhãn dự đoán đúng
- $N$ là tổng số lượng nhãn (tương ứng với số lượng
từ) trong ngữ liệu Gold.<br>

In [41]:
INPUT = input("Test: ")
#Alex was born in California
pos =tagger.tag(INPUT.split())
print("Gold:",gold_data)
count = 0
for i in range(len(pos)):
  if pos[i][1] == gold_data[i][1]:
    count += 1
print("Kết quả dự đoán: ",pos)
print("Accuracy:",count/len(pos))

Test: Alex was born in California
Gold: [('Alex', 'NNP'), ('was', 'NNP'), ('born', 'VB'), ('in', 'NNP'), ('California', 'NNP')]
Kết quả dự đoán:  [('Alex', 'NNP'), ('was', 'NNP'), ('born', 'NNP'), ('in', 'NNP'), ('California', 'NNP')]
Accuracy: 0.8


# **Biểu đồ so sánh đánh giá có và không Laplace**
---

# **XÁC ĐỊNH TỪ GỐC (WORD TOKENIZATION)**
---

# **So sánh kết quả với thư viện Stanford CoreNLP**
---

In [42]:
import stanfordnlp
#stanfordnlp.download("en")
nlp = stanfordnlp.Pipeline(processors='tokenize,mwt,pos,lemma')

Use device: cpu
---
Loading: tokenize
With settings: 
{'model_path': '/root/stanfordnlp_resources/en_ewt_models/en_ewt_tokenizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: pos
With settings: 
{'model_path': '/root/stanfordnlp_resources/en_ewt_models/en_ewt_tagger.pt', 'pretrain_path': '/root/stanfordnlp_resources/en_ewt_models/en_ewt.pretrain.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: lemma
With settings: 
{'model_path': '/root/stanfordnlp_resources/en_ewt_models/en_ewt_lemmatizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
Building an attentional Seq2Seq model...
Using a Bi-LSTM encoder
Using soft attention for LSTM.
Finetune all embeddings.
[Running seq2seq lemmatizer with edit classifier]
Done loading processors!
---


In [43]:
doc = nlp(INPUT)
print(*[f'word: {word.text+" "}\tlemma: {word.lemma}\tPOS: {word.pos}' for sent in doc.sentences for word in sent.words], sep='\n')
#print(*[f'word: {word.text+" "}\tPOS: {word.pos}' for sent in doc.sentences for word in sent.words], sep='\n')

word: Alex 	lemma: Alex	POS: NNP
word: was 	lemma: be	POS: VBD
word: born 	lemma: bear	POS: VBN
word: in 	lemma: in	POS: IN
word: California 	lemma: California	POS: NNP
